## Let's start with basic derivative functions

In [1]:
def dy_dx(x):
  return 2*x

In [2]:
dy_dx(3)


6

## Let's use pytorch lib to use derivatives

In [3]:
import torch

# require_grad=True means, it will track the variable x for coming derivatives function. Note: it's not derivative the value x itself.
x = torch.tensor(3.0, requires_grad=True)

In [4]:
y = x ** 2

# Note: If take derivative of x**2, then y will 2x.

In [5]:
x

tensor(3., requires_grad=True)

In [6]:
# In backend, calculation is happening like this: forward direction: x -> func(square(x)) -> y. This is the forward direction. We are using require_grad = true to remember the values.
y

tensor(9., grad_fn=<PowBackward0>)

In [7]:
# In backend, calculation is happening like this: backward direction: x <- dy/dx of func(square(x)) <- y. This is the backward direction. Final value of x after backward direction will 6
y.backward()

In [8]:
y

tensor(9., grad_fn=<PowBackward0>)

In [9]:
# result of derivative w.r.t x is store using grad
x.grad

tensor(6.)

## Let's use another example: dz/dx = dy/dx * dz/dy

In [11]:
# This is the derivative function without the use of pytorch.
import math

def dz_dx(x):
    return 2 * x * math.cos(x**2)

dz_dx(4)

-7.661275842587077

In [12]:
# Now, let's use torch lib
x = torch.tensor(4.0, requires_grad=True)

In [13]:
y = x ** 2
z = torch.sin(y)

In [14]:
x

tensor(4., requires_grad=True)

In [15]:
y

tensor(16., grad_fn=<PowBackward0>)

In [16]:
z

tensor(-0.2879, grad_fn=<SinBackward0>)

In [17]:
z.backward()

In [18]:
x.grad

tensor(-7.6613)

### See, we have same values of x with backward function and without backward function

### dz_dx = -7.661275... x.grad = -7.6613

# Now, we will build a basic and simple model with autograd and without autograd feature.

In [19]:
# Inputs
x = torch.tensor(6.7)  # Input feature
y = torch.tensor(0.0)  # True label (binary)

# Randomly select values of w and b. It could be any numbers just for fun.
w = torch.tensor(1.0)  # Weight
b = torch.tensor(0.0)  # Bias

In [20]:
# Forward pass

z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

In [21]:
#Compute loss using  Binary Cross-Entropy Loss
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))


In [23]:
# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012)

In [24]:
# Backward pass: Derivatives

# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [25]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


In [26]:
# Now, using with auto-grad

x = torch.tensor(6.7)
y = torch.tensor(0.0)

In [27]:
w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [29]:
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [30]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [31]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [32]:
loss.backward()

In [33]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


# Woow! Using backward and grad, we can easily compute derivatives which is really good.
# Above all examples were scalar means, 1 value. Now, Assume, we have vector
# consist of different numbers.
# Let's see how does it work!

In [34]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

In [35]:
x

tensor([1., 2., 3.], requires_grad=True)

In [36]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [37]:
y.backward()

In [38]:
# Here, we are using the concept of partial derivatives which is mainly use for vectors. Like There are three values (1.0, 2.0, 3.0) in vector.
# Let's them with x1, x2, x3. Now, partial derivative w.r.t x1 will dy/dx1 = (2x1) / 3 , dy/dx2 = (2x2) / 3, dy/dx3 = (2x3) / 3. And 3 is coming from mean function (like average).
# This is how backward works on vector and show values using grad.
x.grad

tensor([0.6667, 1.3333, 2.0000])

In [46]:
# If you run y.backward multiple times, then value of x like x.grad will difference every time like 4, 16, etc which is not good.
# We will use clear_grad to fix the value of x.grad

x = torch.tensor(2.0, requires_grad=True)
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [47]:
y.backward()

In [48]:
x.grad

tensor(4.)

In [50]:
x.grad.zero_()
# After run this cell, x will 0. So, you'll have to run above three cell again to get x.grad value

tensor(0.)

In [51]:
# We can disable tracking of variables.
x = torch.tensor(2.0, requires_grad=True)
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [52]:
y.backward()

In [53]:
x.grad

tensor(4.)

In [55]:
with torch.no_grad():
    y = x ** 2
    y.backward()

# Note: It will cause error, because we are disabling the track graph using torch.no_grad

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn